# Introduction

Some handy examples showing how to automatically download files from the web and Google drive. The examples chosen are for the Save the Children stream of the DataKind 2022 DataDive.

- [project brief](https://docs.google.com/document/d/1TQ2TiGK_k8KEIUPzVb3ZSKaxzEBP9s9ZJBI2U_HeQ6U/edit#)
- [Google drive with data](https://drive.google.com/drive/folders/1G_CAhpb0xV9zRrV-T5ngA03x-IZ4tGKz)

## Setup

In [1]:
# If you don't have these installed, uncomment these lines and run
# %pip install gdown
# %pip install pandas

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 5.2 MB 4.9 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Using cached pytz-2022.6-py2.py3-none-any.whl (498 kB)
     |████████████████████████████████| 10.7 MB 13.1 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done

In [ ]:
import gdown
import os
import pandas as pd

: 

## Examples
### Downloading a single file from GDrive

In [8]:
url_demo_data = "https://docs.google.com/spreadsheets/d/1i41eQRgQatGbchbq2dcL6r9AVyndTSrl/edit?usp=share_link&ouid=106808949113099347741&rtpof=true&sd=true"
filename_demo_data = "kenya-3w-final-list-2017.xslx"
gdown.download(url_demo_data, filename_demo_data, quiet=False)

/Users/matthewharris/Library/Python/3.8/lib/python/site-packages/gdown/parse_url.py:35: UserWarning: You specified a Google Drive link that is not the correct link to download a file. You might want to try `--fuzzy` option or the following url: https://drive.google.com/uc?id=None
  warnings.warn(
Downloading...
From: https://docs.google.com/spreadsheets/d/1i41eQRgQatGbchbq2dcL6r9AVyndTSrl/edit?usp=share_link&ouid=106808949113099347741&rtpof=true&sd=true
To: /Users/matthewharris/Desktop/git/STC_DataDive_Dec22/notebooks/dividor/kenya-3w-final-list-2017.xslx
300kB [00:00, 3.45MB/s]


'kenya-3w-final-list-2017.xslx'

### Downloading a folder from gdrive

Beware, 50 files maximum.

In [9]:
folder_url = "https://drive.google.com/drive/folders/1G_CAhpb0xV9zRrV-T5ngA03x-IZ4tGKz?usp=share_link"
gdown.download_folder(folder_url, quiet=True, use_cookies=False)


['/Users/matthewharris/Desktop/git/STC_DataDive_Dec22/notebooks/dividor/Kenya/dbo-foodconsumptionscores.csv',
 '/Users/matthewharris/Desktop/git/STC_DataDive_Dec22/notebooks/dividor/Kenya/fts_incoming_funding_ken.csv',
 '/Users/matthewharris/Desktop/git/STC_DataDive_Dec22/notebooks/dividor/Kenya/fts_requirements_funding_cluster_ken.csv',
 '/Users/matthewharris/Desktop/git/STC_DataDive_Dec22/notebooks/dividor/Kenya/fts_requirements_funding_covid_ken.csv',
 '/Users/matthewharris/Desktop/git/STC_DataDive_Dec22/notebooks/dividor/Kenya/fts_requirements_funding_globalcluster_ken.csv',
 '/Users/matthewharris/Desktop/git/STC_DataDive_Dec22/notebooks/dividor/Kenya/fts_requirements_funding_ken.csv',
 '/Users/matthewharris/Desktop/git/STC_DataDive_Dec22/notebooks/dividor/Kenya/health_ken.csv',
 '/Users/matthewharris/Desktop/git/STC_DataDive_Dec22/notebooks/dividor/Kenya/hotosm_ken_roads_lines_shp.zip',
 '/Users/matthewharris/Desktop/git/STC_DataDive_Dec22/notebooks/dividor/Kenya/KE_202207.zip',
 

## Reading files in a directory to pandas

This snippet will read all csv and xls files in a directory into pandas dataframes, create a list of them.

Just an example, needs a bit more logic to handle xlsx files with multiple tabs.

In [6]:
dir = "./Kenya"

# Array of pandas dataframes
dfs = {}

for f in os.listdir(dir):
    f = f"{dir}/{f}"
    if '.csv' in f:
        print(f"Loading csv file {f}")
        dfs{f} = pd.read_csv(f)
    if '.xlsx' in f:
        
    

Loading csv file ./Kenya/fts_requirements_funding_covid_ken.csv
Loading csv file ./Kenya/dbo-foodconsumptionscores.csv
Loading csv file ./Kenya/wfp_food_prices_ken.csv
Loading csv file ./Kenya/fts_requirements_funding_cluster_ken.csv
Loading csv file ./Kenya/fts_incoming_funding_ken.csv
Loading csv file ./Kenya/health_ken.csv
Loading csv file ./Kenya/fts_requirements_funding_ken.csv
Loading csv file ./Kenya/ken_admpop_adm1_2019.csv
Loading csv file ./Kenya/ken_admpop_adm0_2019.csv
Loading csv file ./Kenya/fts_requirements_funding_globalcluster_ken.csv
